## Wrangling Data with MongoDB

Goals:
- Connect to a MongoDB Server
- Explore the database and locate PM2.5 readings
- Import database query into DataFrame

In [1]:
from pprint import PrettyPrinter

import pandas as pd
from pymongo import MongoClient

In [3]:
sept_url="https://openafrica.net/dataset/44359020-b2b0-4b66-af09-3de18d6519dc/resource/01e39e28-19c8-405e-b0fd-a1c9f4c4b01d/download/september_2018_sensor_data_archive.csv"
oct_url ="https://openafrica.net/dataset/44359020-b2b0-4b66-af09-3de18d6519dc/resource/53510747-aaff-458b-87a9-66d09c8b37b9/download/october_2018_sensor_data_archive.csv"
nov_url="https://openafrica.net/dataset/44359020-b2b0-4b66-af09-3de18d6519dc/resource/530b63f3-4027-4e95-80cc-6efd4dc45af9/download/november_2018_sensor_data_archive.csv"
dec_ur="https://openafrica.net/dataset/44359020-b2b0-4b66-af09-3de18d6519dc/resource/9d6ce0cd-b6e6-45ff-b6f1-13ba55686d23/download/december_2018_sensor_data_archive.csv"



In [5]:
# sept_df = pd.read_csv(sept_url)
# oct_df = pd.read_csv(oct_url)
# nov_df = pd.read_csv(nov_url)
# dec_df = pd.read_csv(dec_ur)
# df = pd.concat(sept_df,oct_df,nov_df,dec_df)

In [3]:
pp = PrettyPrinter(indent=2)

### Connect

In [5]:
client = MongoClient(host="localhost", port=27017)
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

### Explore

In [ ]:
pp.pprint(list(client.list_databases()))

In [ ]:
# [ {'empty': False, 'name': 'admin', 'sizeOnDisk': 40960},
#   {'empty': False, 'name': 'air-quality', 'sizeOnDisk': 4190208},
#   {'empty': False, 'name': 'config', 'sizeOnDisk': 61440},
#   {'empty': False, 'name': 'local', 'sizeOnDisk': 73728},
#   {'empty': False, 'name': 'wqu-abtest', 'sizeOnDisk': 585728}]

Asign the "air-quality" database to a variable db

In [ ]:
db = client["air-quality"]

List Collections

In [ ]:
list(db.list_collections())[0]

In [9]:
# {'name': 'lagos',
#  'type': 'timeseries',
#  'options': {'timeseries': {'timeField': 'timestamp',
#    'metaField': 'metadata',
#    'granularity': 'seconds',
#    'bucketMaxSpanSeconds': 3600}},
#  'info': {'readOnly': False}}

In [ ]:
for c in db.list_collections():
    print(c["name"])

In [ ]:
# lagos
# system.buckets.lagos
# system.views
# dar-es-salaam
# system.buckets.dar-es-salaam
# nairobi
# system.buckets.nairobi

Asign Collections

In [ ]:
nairobi = db["nairobi"]

Count Documents

In [ ]:
nairobi.count_documents({})
# 202212

Find One Document

In [ ]:
result = nairobi.find_one({})
pp.pprint(result)

In [ ]:
# { '_id': ObjectId('6525d772f44bfedd842a6fcc'),
#   'metadata': { 'lat': -1.3,
#                 'lon': 36.785,
#                 'measurement': 'temperature',
#                 'sensor_id': 58,
#                 'sensor_type': 'DHT22',
#                 'site': 29},
#   'temperature': 16.5,
#   'timestamp': datetime.datetime(2018, 9, 1, 0, 0, 4, 301000)}

Distinct Censor Sites

In [ ]:
nairobi.distinct("metadata.site")
Output:
[6, 29]

Count Documents by Site: Which site has more data?

In [ ]:
print("Documents from site 6:", nairobi.count_documents({"metadata.site": 6}))
print("Documents from site 29:", nairobi.count_documents({"metadata.site": 29}))

Output:
Documents from site 6: 70360
Documents from site 29: 13185

Aggregate Documents

In [10]:
result = nairobi.aggregate(
    [{
        "$group":{"_id": "$metadata.site"}
    }])
pp.pprint(list(result))

333

Distinct Measurements

In [ ]:
nairobi.distinct("metadata.measurement")
['P2', 'humidity', 'P1', 'temperature']

Find PM2.5 Readings

In [ ]:
result = nairobi.find({"metadata.measurement": "P2"}).limit(3)
pp.pprint(list(result))

In [ ]:
[ { 'P2': 34.43,
    '_id': ObjectId('6525d775f44bfedd842bf24d'),
    'metadata': { 'lat': -1.3,
                  'lon': 36.785,
                  'measurement': 'P2',
                  'sensor_id': 57,
                  'sensor_type': 'SDS011',
                  'site': 29},
    'timestamp': datetime.datetime(2018, 9, 1, 0, 0, 2, 472000)},
  { 'P2': 30.53,
    '_id': ObjectId('6525d775f44bfedd842bf24e'),
    'metadata': { 'lat': -1.3,
                  'lon': 36.785,
                  'measurement': 'P2',
                  'sensor_id': 57,
                  'sensor_type': 'SDS011',
                  'site': 29},
    'timestamp': datetime.datetime(2018, 9, 1, 0, 5, 3, 941000)},
  { 'P2': 22.8,
    '_id': ObjectId('6525d775f44bfedd842bf24f'),
    'metadata': { 'lat': -1.3,
                  'lon': 36.785,
                  'measurement': 'P2',
                  'sensor_id': 57,
                  'sensor_type': 'SDS011',
                  'site': 29},

Aggregate by Measurement I

In [ ]:

result = nairobi.aggregate(
    [
        {"$match":{"metadata.site": 6}},  
        {"$group":{"_id": "$metadata.measurement", "count": {"$count": {}}}}
    ]
)
pp.pprint(list(result))

In [ ]:
[ {'_id': 'humidity', 'count': 17011},
  {'_id': 'P1', 'count': 18169},
  {'_id': 'temperature', 'count': 17011},
  {'_id': 'P2', 'count': 18169}]

Aggregate by Measurement II

In [ ]:

result = nairobi.aggregate(
    [
        {"$match":{"metadata.site": 29}},  
        {"$group":{"_id": "$metadata.measurement", "count": {"$count": {}}}}
    ]
)
pp.pprint(list(result))

In [ ]:
[ {'_id': 'P1', 'count': 32907},
  {'_id': 'humidity', 'count': 33019},
  {'_id': 'temperature', 'count': 33019},
  {'_id': 'P2', 'count': 32907}]

### Import

Find by Site and Measurement

In [ ]:
result = nairobi.find(
    {"metadata.site" : 29, "metadata.measurement": "P2"},
    projection={"P2": 1, "timestamp": 1, "_id": 0}
)
pp.pprint(result.next())

In [ ]:
{'P2': 34.43, 'timestamp': datetime.datetime(2018, 9, 1, 0, 0, 2, 472000)}

Read Query Into DataFrame

In [ ]:
df = pd.DataFrame(result).set_index("timestamp")
df.head()